# Flight Delay Prediction

the goal of this project is to predict the flight that will be delayed for over 15 mins in January for US flights.

We will read the data first and remove unneeded features, hence the data contains the diverted and cancelled flights.

In [2]:
#importing libraries that we will using during this project
import numpy as np
import pandas as pd

We will import flights data for January 2019 and January 2020, then we will concatenate dataframes together

In [9]:
#reading data and assign it to variables
jan19data = pd.read_csv('data/Jan_2019_ontime.csv')
jan20data = pd.read_csv('data/Jan_2020_ontime.csv')

#concatenate dataframes 
df = pd.concat([jan19data , jan20data])

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1191331 entries, 0 to 607345
Data columns (total 22 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   DAY_OF_MONTH           1191331 non-null  int64  
 1   DAY_OF_WEEK            1191331 non-null  int64  
 2   OP_UNIQUE_CARRIER      1191331 non-null  object 
 3   OP_CARRIER_AIRLINE_ID  1191331 non-null  int64  
 4   OP_CARRIER             1191331 non-null  object 
 5   TAIL_NUM               1188090 non-null  object 
 6   OP_CARRIER_FL_NUM      1191331 non-null  int64  
 7   ORIGIN_AIRPORT_ID      1191331 non-null  int64  
 8   ORIGIN_AIRPORT_SEQ_ID  1191331 non-null  int64  
 9   ORIGIN                 1191331 non-null  object 
 10  DEST_AIRPORT_ID        1191331 non-null  int64  
 11  DEST_AIRPORT_SEQ_ID    1191331 non-null  int64  
 12  DEST                   1191331 non-null  object 
 13  DEP_TIME               1168315 non-null  float64
 14  DEP_DEL15          

# Data Summary
our data now have 22 columns and 1191331 rows

## dataframe columns will use (14 columns):

* 'DAY_OF_MONTH': day of month.
* 'DAY_OF_WEEK': day number starting from monday.
* 'OP_UNIQUE_CARRIER': carrier identifier.
* 'TAIL_NUM': tail number.
* 'OP_CARRIER_FL_NUM': flight number.
* 'DEP_TIME_BLK': 24h time chunks.
* 'ORIGIN': departure airport identifier.
* 'DEST': destination airport identfier.
* 'DEP_TIME': actual departure time.
* 'DEP_DEL15': departure Delay Indicator, 15 Minutes or More. boolean
* 'DEP_TIME_BLK': departure Time Block, hourly intervals.
* 'ARR_TIME': actual arrival time.
* 'ARR_DEL15': arrival delay indicator, 15 Minutes or More. boolean
* 'DISTANCE': flight length.


## columns will drop (6 columns):
* 'OP_CARRIER_AIRLINE_ID': 
* 'OP_CARRIER':
* 'ORIGIN_AIRPORT_SEQ_ID':
* 'DEST_AIRPORT_SEQ_ID':
* 'CANCELLED':
* 'DIVERTED':



We will start by droping unneeded columns